<a href="https://colab.research.google.com/github/fred-creator-creat/oficina-sql-database-design/blob/main/Automotive_Service_System_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Projeto: Sistema de Gerenciamento de Oficina Mecânica
Desafio de Projeto: Esquema Lógico e Implementação de Banco de Dados SQL
Contexto: Bootcamp / Disciplina de Modelagem de Dados
Ferramentas: MySQL, Python, Google Colab, Diagrams.io

In [8]:
# 1. Instala o servidor e o conector
!apt-get update
!apt-get install mysql-server > /dev/null
!pip install mysql-connector-python > /dev/null

# 2. Inicia o serviço
!service mysql start

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://cli.github.com/packages stable/main amd64 Packages [355 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,904 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,749 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,756 kB]
Get:14 htt

In [9]:
# Comando para mudar o método de autenticação do Root
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY '';"
!mysql -e "FLUSH PRIVILEGES;"

In [10]:
import mysql.connector

# 1. Conectar ao servidor local do Colab
db_setup = mysql.connector.connect(
  host="localhost",
  user="root",
  password=""
)
cursor = db_setup.cursor()

# 2. Criar o banco de dados do projeto Oficina e selecionar para uso
cursor.execute("DROP DATABASE IF EXISTS oficina_db;") # Limpa se já existir
cursor.execute("CREATE DATABASE oficina_db;")
cursor.execute("USE oficina_db;")

print("Banco de dados 'oficina_db' criado e selecionado com sucesso!")

Banco de dados 'oficina_db' criado e selecionado com sucesso!


In [11]:
import pandas as pd
from sqlalchemy import create_engine

# Criando a engine de conexão para o banco oficina_db
# O Pandas usará isso para mostrar as tabelas de forma legível
engine = create_engine('mysql+mysqlconnector://root@localhost/oficina_db')

# Ativa a visualização de tabelas interativas do Colab
%load_ext google.colab.data_table

print("Mecanismo de visualização Pandas/SQLAlchemy configurado para o banco 'oficina_db'.")

Mecanismo de visualização Pandas/SQLAlchemy configurado para o banco 'oficina_db'.


In [12]:
# 1. Tabela de Cliente
cursor.execute("""
CREATE TABLE IF NOT EXISTS client (
    idClient INT AUTO_INCREMENT PRIMARY KEY,
    fname VARCHAR(50) NOT NULL,
    lname VARCHAR(50),
    cpf CHAR(11) NOT NULL,
    address VARCHAR(255),
    contact CHAR(11),
    CONSTRAINT unique_cpf_client UNIQUE (cpf)
);
""")

# 2. Tabela de Veículo
cursor.execute("""
CREATE TABLE IF NOT EXISTS vehicle (
    idVehicle INT AUTO_INCREMENT PRIMARY KEY,
    idVehicleClient INT,
    plate CHAR(7) NOT NULL,
    model VARCHAR(45),
    brand VARCHAR(45),
    CONSTRAINT unique_plate_vehicle UNIQUE (plate),
    CONSTRAINT fk_vehicle_client FOREIGN KEY (idVehicleClient) REFERENCES client(idClient)
        ON UPDATE CASCADE ON DELETE CASCADE
);
""")

# 3. Tabela de Mecânico
cursor.execute("""
CREATE TABLE IF NOT EXISTS mechanic (
    idMechanic INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    address VARCHAR(255),
    specialty VARCHAR(45)
);
""")

db_setup.commit()
print("Tabelas 'client', 'vehicle' e 'mechanic' criadas com sucesso!")

Tabelas 'client', 'vehicle' e 'mechanic' criadas com sucesso!


In [13]:
# 1. Tabela de Ordem de Serviço (OS)
cursor.execute("""
CREATE TABLE IF NOT EXISTS service_order (
    idOS INT AUTO_INCREMENT PRIMARY KEY,
    idOSVehicle INT,
    issue_date DATE NOT NULL,
    total_amount FLOAT DEFAULT 0,
    os_status ENUM('Aberto', 'Em análise', 'Aprovado', 'Concluído', 'Cancelado') DEFAULT 'Aberto',
    delivery_date DATE,
    CONSTRAINT fk_os_vehicle FOREIGN KEY (idOSVehicle) REFERENCES vehicle(idVehicle)
        ON UPDATE CASCADE
);
""")

# 2. Tabela de Serviços (Catálogo de serviços disponíveis)
cursor.execute("""
CREATE TABLE IF NOT EXISTS service (
    idService INT AUTO_INCREMENT PRIMARY KEY,
    description VARCHAR(255) NOT NULL,
    service_value FLOAT NOT NULL
);
""")

# 3. Tabela de Peças (Catálogo de peças)
cursor.execute("""
CREATE TABLE IF NOT EXISTS part (
    idPart INT AUTO_INCREMENT PRIMARY KEY,
    description VARCHAR(255) NOT NULL,
    part_value FLOAT NOT NULL
);
""")

# 4. Tabela de Ligação: OS e Serviços (N:N)
cursor.execute("""
CREATE TABLE IF NOT EXISTS os_has_services (
    idOS_service INT,
    idService_os INT,
    quantity INT DEFAULT 1,
    PRIMARY KEY (idOS_service, idService_os),
    CONSTRAINT fk_os_has_services_os FOREIGN KEY (idOS_service) REFERENCES service_order(idOS),
    CONSTRAINT fk_os_has_services_service FOREIGN KEY (idService_os) REFERENCES service(idService)
);
""")

# 5. Tabela de Ligação: OS e Peças (N:N)
cursor.execute("""
CREATE TABLE IF NOT EXISTS os_has_parts (
    idOS_part INT,
    idPart_os INT,
    quantity INT DEFAULT 1,
    PRIMARY KEY (idOS_part, idPart_os),
    CONSTRAINT fk_os_has_parts_os FOREIGN KEY (idOS_part) REFERENCES service_order(idOS),
    CONSTRAINT fk_os_has_parts_part FOREIGN KEY (idPart_os) REFERENCES part(idPart)
);
""")

db_setup.commit()
print("Tabelas de OS, Serviços e Peças criadas com sucesso!")

Tabelas de OS, Serviços e Peças criadas com sucesso!


In [14]:
# Validação Final da Estrutura do Banco de Dados Oficina
print("--- ESTRUTURA FINAL DO BANCO DE DADOS: OFICINA ---")
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()

for (table,) in tables:
    print(f"Tabela Confirmada: {table}")

print(f"\nTotal de tabelas criadas: {len(tables)}")
print("Status: Esperado 8 tabelas (client, mechanic, os_has_parts, os_has_services, part, service, service_order, vehicle).")
print("--------------------------------------------------")

--- ESTRUTURA FINAL DO BANCO DE DADOS: OFICINA ---
Tabela Confirmada: client
Tabela Confirmada: mechanic
Tabela Confirmada: os_has_parts
Tabela Confirmada: os_has_services
Tabela Confirmada: part
Tabela Confirmada: service
Tabela Confirmada: service_order
Tabela Confirmada: vehicle

Total de tabelas criadas: 8
Status: Esperado 8 tabelas (client, mechanic, os_has_parts, os_has_services, part, service, service_order, vehicle).
--------------------------------------------------


In [15]:
# --- INSERÇÃO DE DADOS PARA TESTE (POPULANDO O BANCO) ---

# 1. Inserindo Clientes
cursor.execute("""
INSERT INTO client (fname, lname, cpf, address, contact) VALUES
('João', 'Silva', '12345678901', 'Rua das Flores, 10 - Centro', '11988887777'),
('Ricardo', 'Oliveira', '98765432100', 'Av. Paulista, 1500 - SP', '11977776666'),
('Beatriz', 'Souza', '45678912344', 'Rua Chile, 45 - Curitiba', '41966665555');
""")

# 2. Inserindo Veículos
cursor.execute("""
INSERT INTO vehicle (idVehicleClient, plate, model, brand) VALUES
(1, 'ABC1234', 'Civic', 'Honda'),
(2, 'XYZ5678', 'Uno', 'Fiat'),
(3, 'KJH9090', 'Corolla', 'Toyota');
""")

# 3. Inserindo Mecânicos
cursor.execute("""
INSERT INTO mechanic (name, address, specialty) VALUES
('Carlos Augusto', 'Rua dos Motores, 100', 'Motor e Câmbio'),
('Marcos Souza', 'Rua das Peças, 50', 'Suspensão e Freios'),
('Juliana Lima', 'Av. Elétrica, 300', 'Elétrica e Injeção');
""")

# 4. Inserindo Catálogo de Serviços e Peças
cursor.execute("INSERT INTO service (description, service_value) VALUES ('Troca de Óleo', 150.00), ('Alinhamento', 80.00), ('Revisão Geral', 450.00);")
cursor.execute("INSERT INTO part (description, part_value) VALUES ('Filtro de Óleo', 45.00), ('Pastilha de Freio', 120.00), ('Óleo 5W30', 200.00);")

# 5. Criando Ordens de Serviço (OS)
cursor.execute("""
INSERT INTO service_order (idOSVehicle, issue_date, total_amount, os_status, delivery_date) VALUES
(1, '2026-02-01', 395.00, 'Concluído', '2026-02-05'),
(2, '2026-02-10', 80.00, 'Em análise', NULL),
(3, '2026-02-15', 0.00, 'Aberto', NULL);
""")

# 6. Vinculando Serviços e Peças às OS (Tabelas de Ligação)
# OS 1: Troca de óleo (Serviço 1) + Filtro e Óleo (Peças 1 e 3)
cursor.execute("INSERT INTO os_has_services (idOS_service, idService_os, quantity) VALUES (1, 1, 1);")
cursor.execute("INSERT INTO os_has_parts (idOS_part, idPart_os, quantity) VALUES (1, 1, 1), (1, 3, 1);")

# OS 2: Apenas Alinhamento (Serviço 2)
cursor.execute("INSERT INTO os_has_services (idOS_service, idService_os, quantity) VALUES (2, 2, 1);")

db_setup.commit()
print("Sucesso! O banco de dados foi populado com dados de teste.")

Sucesso! O banco de dados foi populado com dados de teste.


In [16]:
print("--- QUERY 1: Listagem de Clientes e seus Veículos (JOIN e ORDER BY) ---")
# Objetivo: Saber quem são os donos dos carros que estão na oficina, ordenados por nome.
query1 = """
SELECT c.fname AS Nome, c.lname AS Sobrenome, v.model AS Modelo, v.plate AS Placa
FROM client c
INNER JOIN vehicle v ON c.idClient = v.idVehicleClient
ORDER BY c.fname;
"""
df1 = pd.read_sql(query1, engine)
display(df1)

--- QUERY 1: Listagem de Clientes e seus Veículos (JOIN e ORDER BY) ---


,Nome,Sobrenome,Modelo,Placa
0,Beatriz,Souza,Corolla,KJH9090
1,João,Silva,Civic,ABC1234
2,Ricardo,Oliveira,Uno,XYZ5678


In [17]:
print("\n--- QUERY 2: OS com Valor Total superior a R$ 100,00 (WHERE e Atributo Derivado) ---")
# Objetivo: Filtrar ordens caras e mostrar quanto falta para a OS ser concluída (Exemplo de lógica de negócio).
# Atributo derivado: 'Status_Financeiro' baseado no valor.
query2 = """
SELECT idOS, total_amount, os_status,
       CASE
           WHEN total_amount > 300 THEN 'Alto Investimento'
           ELSE 'Manutenção Comum'
       END AS categoria_valor
FROM service_order
WHERE total_amount > 100;
"""
df2 = pd.read_sql(query2, engine)
display(df2)


--- QUERY 2: OS com Valor Total superior a R$ 100,00 (WHERE e Atributo Derivado) ---


,idOS,total_amount,os_status,categoria_valor
0,1,395.0,Concluído,Alto Investimento


In [18]:
print("\n--- QUERY 3: Quantidade de Ordens por Status (GROUP BY e HAVING) ---")
# Objetivo: Contar quantas OS existem por status, mas só mostrar os status que possuem mais de 0 registros (filtros de grupo).
query3 = """
SELECT os_status, COUNT(*) AS total_por_status
FROM service_order
GROUP BY os_status
HAVING COUNT(*) > 0;
"""
df3 = pd.read_sql(query3, engine)
display(df3)


--- QUERY 3: Quantidade de Ordens por Status (GROUP BY e HAVING) ---


,os_status,total_por_status
0,Concluído,1
1,Em análise,1
2,Aberto,1


In [19]:
print("\n--- QUERY 4: Relatório Completo - Cliente, Veículo e Peças Utilizadas (Múltiplos JOINs) ---")
# Objetivo: Cruzar quase todas as tabelas para ver qual peça foi usada no carro de qual cliente.
query4 = """
SELECT c.fname AS Cliente, v.model AS Carro, p.description AS Peça, op.quantity AS Qtd
FROM client c
JOIN vehicle v ON c.idClient = v.idVehicleClient
JOIN service_order so ON v.idVehicle = so.idOSVehicle
JOIN os_has_parts op ON so.idOS = op.idOS_part
JOIN part p ON op.idPart_os = p.idPart;
"""
df4 = pd.read_sql(query4, engine)
display(df4)


--- QUERY 4: Relatório Completo - Cliente, Veículo e Peças Utilizadas (Múltiplos JOINs) ---


,Cliente,Carro,Peça,Qtd
0,João,Civic,Filtro de Óleo,1
1,João,Civic,Óleo 5W30,1


In [20]:
print("--- QUERY 5: Faturamento por Cliente (Agregação e JOIN) ---")
# Objetivo: Saber quem são os clientes que mais trazem retorno financeiro para a oficina.
query5 = """
SELECT c.fname AS Cliente, SUM(so.total_amount) AS Total_Gasto
FROM client c
JOIN vehicle v ON c.idClient = v.idVehicleClient
JOIN service_order so ON v.idVehicle = so.idOSVehicle
GROUP BY c.idClient
ORDER BY Total_Gasto DESC;
"""
df5 = pd.read_sql(query5, engine)
display(df5)

--- QUERY 5: Faturamento por Cliente (Agregação e JOIN) ---


,Cliente,Total_Gasto
0,João,395.0
1,Ricardo,80.0
2,Beatriz,0.0


In [21]:
print("\n--- QUERY 6: Detalhamento de Mão de Obra vs Peças (Atributos Derivados Complexos) ---")
# Objetivo: Mostrar a composição de custo de cada OS concluída.
query6 = """
SELECT
    idOS,
    issue_date,
    total_amount AS Valor_Total,
    ROUND(total_amount * 0.7, 2) AS Estimativa_Custo_Pecas,
    ROUND(total_amount * 0.3, 2) AS Estimativa_Lucro_Mao_Obra
FROM service_order
WHERE os_status = 'Concluído';
"""
df6 = pd.read_sql(query6, engine)
display(df6)


--- QUERY 6: Detalhamento de Mão de Obra vs Peças (Atributos Derivados Complexos) ---


,idOS,issue_date,Valor_Total,Estimativa_Custo_Pecas,Estimativa_Lucro_Mao_Obra
0,1,2026-02-01,395.0,276.5,118.5


In [22]:
print("\n--- QUERY 7: Serviços mais solicitados (Ordenação e Limite) ---")
# Objetivo: Identificar qual serviço é o "carro-chefe" da oficina.
query7 = """
SELECT s.description AS Servico, COUNT(hs.idService_os) AS Qtd_Execucoes
FROM service s
JOIN os_has_services hs ON s.idService = hs.idService_os
GROUP BY s.idService
ORDER BY Qtd_Execucoes DESC
LIMIT 5;
"""
df7 = pd.read_sql(query7, engine)
display(df7)


--- QUERY 7: Serviços mais solicitados (Ordenação e Limite) ---


,Servico,Qtd_Execucoes
0,Troca de Óleo,1
1,Alinhamento,1
